In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')


KeyboardInterrupt



In [ ]:
sns.set()
sns.set_style("whitegrid")
sns.set_context('talk')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set(font='SimHei')

In [ ]:
data_train = pd.read_csv('train.csv')
data_test_a = pd.read_csv('testA.csv')

In [ ]:
data_train.isnull().sum()

In [ ]:
print('Train data shape:', data_train.shape)
print('TestA data shape:', data_test_a.shape)

In [ ]:
data_train.head(5)

In [ ]:
data_train.columns

In [ ]:
data_train.info()

In [ ]:
data_train.describe()

In [ ]:
data_train.isnull().any()

In [ ]:
print(f'There are {data_train.isnull().any().sum()}column feature data missing')

In [ ]:
have_null_fea_dict = (data_train.isnull().sum() / len(data_train)).to_dict()
fea_null_moreThanHalf = {}
for key, value in have_null_fea_dict.items():
    if value > 0.5: 
        fea_null_moreThanHalf[key] = value
fea_null_moreThanHalf

In [ ]:
missing = data_train.isnull().sum() / len(data_train) 
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [ ]:
one_value_fea = [
    col for col in data_train.columns if data_train[col].nunique() <= 1
]
one_value_fea_test = [
    col for col in data_test_a.columns if data_test_a[col].nunique() <= 1
]
print("The feature attribute in the training set has only one value:", one_value_fea)
print("Feature attributes in the test set have only one-value features:", one_value_fea_test)

In [ ]:
numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(
    filter(lambda x: x not in numerical_fea, list(data_train.columns)))

In [ ]:
numerical_fea

In [ ]:
category_fea

In [ ]:
data_train.grade 

In [ ]:
def get_numerical_serial_fea(data, feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique() 
        if temp <= 10:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea, numerical_noserial_fea


numerical_serial_fea, numerical_noserial_fea = get_numerical_serial_fea(
    data_train, numerical_fea)

In [ ]:
numerical_serial_fea

In [ ]:
numerical_noserial_fea

In [ ]:
data_train['term'].value_counts()

In [ ]:
data_train['homeOwnership'].value_counts()

In [ ]:
f = pd.melt(data_train, value_vars=numerical_serial_fea)
g = sns.FacetGrid(f, col="variable", col_wrap=2, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

In [ ]:
plt.figure(figsize=(16, 12))
plt.suptitle('Transaction Values Distribution', fontsize=22)
plt.subplot(221)
sub_plot_1 = sns.distplot(data_train['loanAmnt'])
sub_plot_1.set_title("loanAmnt Distribuition", fontsize=18)
sub_plot_1.set_xlabel("")
sub_plot_1.set_ylabel("Probability", fontsize=15)

plt.subplot(222)
sub_plot_2 = sns.distplot(np.log(data_train['loanAmnt']))
sub_plot_2.set_title("loanAmnt (Log) Distribuition", fontsize=18)
sub_plot_2.set_xlabel("")
sub_plot_2.set_ylabel("Probability", fontsize=15)

In [ ]:
data_train['employmentLength'].value_counts()

In [ ]:
data_train['issueDate'].value_counts()

In [ ]:
data_train['earliesCreditLine'].value_counts()

In [ ]:
data_train['isDefault'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')

f, ax = plt.subplots(1, 2, figsize=(10, 6))
data_train['isDefault'].value_counts().plot.pie(
    explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('isDefault')

sns.countplot('isDefault', data=data_train, ax=ax[1])

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(
    data_train["employmentLength"].value_counts(dropna=False)[:20],
    data_train["employmentLength"].value_counts(dropna=False).keys()[:20])
plt.show()

In [ ]:
train_loan_fr = data_train.loc[data_train['isDefault'] == 1]
train_loan_nofr = data_train.loc[data_train['isDefault'] == 0]
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 8))
train_loan_fr.groupby('grade')['grade'].count().plot(
    kind='barh', ax=ax1, title='Count of grade fraud')
train_loan_nofr.groupby('grade')['grade'].count().plot(
    kind='barh', ax=ax2, title='Count of grade non-fraud')
train_loan_fr.groupby('employmentLength')['employmentLength'].count().plot(
    kind='barh', ax=ax3, title='Count of employmentLength fraud')
train_loan_nofr.groupby('employmentLength')['employmentLength'].count().plot(
    kind='barh', ax=ax4, title='Count of employmentLength non-fraud')
plt.show()

In [ ]:
fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(15, 6))
data_train.loc[data_train['isDefault'] == 1] \
    ['loanAmnt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log Loan Amt - Fraud',
          color='r',
          xlim=(-3, 10),
         ax= ax1)
data_train.loc[data_train['isDefault'] == 0] \
    ['loanAmnt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log Loan Amt - Not Fraud',
          color='b',
          xlim=(-3, 10),
         ax=ax2)

In [ ]:
total = len(data_train)
total_amt = data_train.groupby(['isDefault'])['loanAmnt'].sum().sum()  
plt.figure(figsize=(12, 5))
plt.subplot(121)  
plot_tr = sns.countplot(
    x='isDefault', data=data_train) 
plot_tr.set_title(
    "Fraud Loan Distribution \n 0: good user | 1: bad user", fontsize=14)
plot_tr.set_xlabel("Is fraud by count", fontsize=16)
plot_tr.set_ylabel('Count', fontsize=16)
for p in plot_tr.patches:
    height = p.get_height()
    plot_tr.text(
        p.get_x() + p.get_width() / 2.,
        height + 3,
        '{:1.2f}%'.format(height / total * 100),
        ha="center",
        fontsize=15)

percent_amt = (data_train.groupby(['isDefault'])['loanAmnt'].sum())
percent_amt = percent_amt.reset_index()
plt.subplot(122)
plot_tr_2 = sns.barplot(
    x='isDefault', y='loanAmnt', dodge=True, data=percent_amt)
plot_tr_2.set_title(
    "Total Amount in loanAmnt  \n 0: good user | 1: bad user", fontsize=14)
plot_tr_2.set_xlabel("Is fraud by percent", fontsize=16)
plot_tr_2.set_ylabel('Total Loan Amount Scalar', fontsize=16)
for p in plot_tr_2.patches:
    height = p.get_height()
    plot_tr_2.text(
        p.get_x() + p.get_width() / 2.,
        height + 3,
        '{:1.2f}%'.format(height / total_amt * 100),
        ha="center",
        fontsize=15)

In [ ]:
data_train['issueDate'] = pd.to_datetime(data_train['issueDate'],format='%Y-%m-%d')
startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
data_train['issueDateDT'] = data_train['issueDate'].apply(lambda x: x-startdate).dt.days

In [ ]:
data_test_a['issueDate'] = pd.to_datetime(data_train['issueDate'],format='%Y-%m-%d')
startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
data_test_a['issueDateDT'] = data_test_a['issueDate'].apply(lambda x: x-startdate).dt.days

In [ ]:
plt.hist(data_train['issueDateDT'], label='train')
plt.hist(data_test_a['issueDateDT'], label='test')
plt.legend()
plt.title('Distribution of issueDateDT dates')

In [ ]:
pivot = pd.pivot_table(
    data_train,
    index=['grade'],
    columns=['issueDateDT'],
    values=['loanAmnt'],
    aggfunc=np.sum)

In [ ]:
pivot

In [ ]:
import pandas_profiling

In [ ]:
pfr = pandas_profiling.ProfileReport(data_train)
pfr.to_file("./data visualization.html")

In [ ]:
numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))
label = 'isDefault'
numerical_fea.remove(label)

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train[numerical_fea] = data_train[numerical_fea].fillna(
    data_train[numerical_fea].median())
data_test_a[numerical_fea] = data_test_a[numerical_fea].fillna(
    data_train[numerical_fea].median())

data_train[category_fea] = data_train[category_fea].fillna(
    data_train[category_fea].mode())
data_test_a[category_fea] = data_test_a[category_fea].fillna(
    data_train[category_fea].mode())

In [ ]:
data_train.isnull().sum()

In [ ]:
category_fea

In [ ]:
for data in [data_train, data_test_a]:
    data['issueDate'] = pd.to_datetime(data['issueDate'], format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x - startdate).dt.days

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])


for data in [data_train, data_test_a]:
    data['employmentLength'].replace(
        to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(
        employmentLength_to_int)

In [ ]:
data_train['employmentLength'].value_counts(dropna=False).sort_index()

In [ ]:
data_train['employmentLength'].value_counts(dropna=False).sort_index()

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train['employmentLength'] = data_train['employmentLength'].fillna(10)

data_test_a['employmentLength'] = data_test_a['employmentLength'].fillna(10)

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train['earliesCreditLine'].sample(5)

In [ ]:
for data in [data_train, data_test_a]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

In [ ]:
data_train['earliesCreditLine'].sample(5)

In [ ]:
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
for f in cate_features:
    print(f, 'Type number：', data[f].nunique())

In [ ]:
from sklearn import preprocessing 

lbl = preprocessing.LabelEncoder()
data_train['grade']=lbl.fit_transform(data_train['grade'])  
data_test_a['grade']=lbl.fit_transform(data_test_a['grade'])  
data_train['subGrade']=lbl.fit_transform(data_train['subGrade'])  
data_test_a['subGrade']=lbl.fit_transform(data_test_a['subGrade']) 

In [ ]:
data_train.info()

In [ ]:
dummy = pd.get_dummies(data_train[['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode']])
data_train = pd.concat([data_train,dummy], axis = 1)
data_train.drop(['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], inplace=True, axis = 1)
data_train.head()

In [ ]:
dummy = pd.get_dummies(data_test_a[['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode']])
data_test_a = pd.concat([data_test_a,dummy], axis = 1)
data_test_a.drop(['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], inplace=True, axis = 1)
data_test_a.head()

In [ ]:
def find_outliers_by_3segama(data,fea):
    data_std = np.std(data[fea])
    data_mean = np.mean(data[fea])
    outliers_cut_off = data_std * 3
    lower_rule = data_mean - outliers_cut_off
    upper_rule = data_mean + outliers_cut_off
    data[fea+'_outliers'] = data[fea].apply(lambda x:str('abnormal value') if x > upper_rule or x < lower_rule else 'normal value')
    return data

In [ ]:
for fea in numerical_fea:
    data_train = find_outliers_by_3segama(data_train,fea)
    print(data_train[fea+'_outliers'].value_counts())
    print(data_train.groupby(fea+'_outliers')['isDefault'].sum())
    print('='*30)

In [ ]:
for fea in numerical_fea:
    data_train = data_train[data_train[fea+'_outliers']=='normal value']
    data_train = data_train.reset_index(drop=True) 

In [ ]:
data_train.iloc[:,0:48]

In [ ]:
x_train = data_train.drop(['isDefault','issueDate','id'], axis=1)

data_corr = x_train.corrwith(data_train.isDefault)
result = pd.DataFrame(columns=['features', 'corr'])
result['features'] = data_corr.index
result['corr'] = data_corr.values

In [ ]:
result['corr'].sort_index()

In [ ]:
data_train = data_train.drop(['issueDate','id','policyCode'], axis=1)
data_test_a = data_test_a.drop(['issueDate','id','policyCode'], axis=1)

In [ ]:
features = [
    f for f in data_train.columns
    if f not in ['isDefault'] and '_outliers' not in f
]
x_train = data_train[features]
x_test = data_test_a[features]
y_train = data_train['isDefault']

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc_z.fit_transform(x_test)

In [ ]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics


X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    x_train, y_train, test_size=0.2, random_state=1234)

AdaBoost1 = ensemble.AdaBoostClassifier()
AdaBoost1.fit(X_train, Y_train)
pred1 = AdaBoost1.predict(X_test)

print('Prediction accuracy of the model in the test set：\n', metrics.accuracy_score(Y_test, pred1))
print('Evaluation report of the model：\n', metrics.classification_report(Y_test, pred1))
print(pred1)

In [ ]:
y_score = AdaBoost1.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)
roc_auc = metrics.auc(fpr,tpr)

plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.5, edgecolor = 'black')

plt.plot(fpr, tpr, color='black', lw = 1)

plt.plot([0,1],[0,1], color = 'red', linestyle = '--')

plt.text(0.5,0.3,'ROC curve (area = %0.2f)' % roc_auc)

plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')

plt.show()
roc_auc

In [ ]:
y_score = AdaBoost1.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)
plt.figure(figsize=(8, 8))
plt.title('Validation ROC')
plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.3, edgecolor = 'black')
plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
plt.ylim(0,1)
plt.xlim(0,1)
plt.legend(loc='best')
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.plot([0,1],[0,1],'r--')
plt.show()

In [ ]:
importance = pd.Series(AdaBoost1.feature_importances_, index = x_train.columns)
plt.figure(figsize=(8, 16))
importance.sort_values().plot(kind = 'barh')
plt.show()
importance.sort_values(ascending = False).head(10).plot(kind = 'barh')
plt.show()

In [ ]:
import numpy as np

pd.set_option('display.max_columns', 20)

pd.set_option('display.max_rows', 130)
important_features = pd.DataFrame({'feature':x_train.columns,'importance':AdaBoost1.feature_importances_})
important_features.sort_values(by = 'importance',ascending = False,inplace =True)
important_features['cum_importance'] = np.cumsum(important_features['importance'])
important_features.head(10)

In [ ]:

from sklearn import metrics
from sklearn import tree

from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics


X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    x_train, y_train, test_size=0.2, random_state=1234)

CART_Class = tree.DecisionTreeClassifier(max_depth=8, min_samples_leaf = 2, min_samples_split=4)

decision_tree = CART_Class.fit(X_train, Y_train)

pred = CART_Class.predict(X_test)

print('Prediction accuracy of the model in the test set:\n',metrics.accuracy_score(Y_test, pred))
print('Evaluation report of the model：\n',metrics.classification_report(Y_test, pred))

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus
from six import StringIO
feature_names=x_train.columns
dot_data = tree.export_graphviz(CART_Class, out_file=None, 
                         feature_names=feature_names,  
                         class_names=['0','1'],  
                         filled=True, rounded=True, special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
y_score = decision_tree.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)


plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.5, edgecolor = 'black')

plt.plot(fpr, tpr, color='black', lw = 1)

plt.plot([0,1],[0,1], color = 'red', linestyle = '--')

plt.text(0.5,0.3,'ROC curve (area = %0.2f)' % roc_auc)

plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')

plt.show()
roc_auc

In [ ]:
y_score = decision_tree.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)
plt.figure(figsize=(8, 8))
plt.title('Validation ROC')
plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.3, edgecolor = 'black')
plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
plt.ylim(0,1)
plt.xlim(0,1)
plt.legend(loc='best')
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.plot([0,1],[0,1],'r--')
plt.show()

In [ ]:
importance = pd.Series(CART_Class.feature_importances_, index = x_train.columns)
plt.figure(figsize=(8, 16))
importance.sort_values().plot(kind = 'barh')
plt.show()
importance.sort_values(ascending = False).head(10).plot(kind = 'barh')
plt.show()

In [ ]:
import numpy as np

pd.set_option('display.max_columns', 20)

pd.set_option('display.max_rows', 130)
important_features = pd.DataFrame({'feature':x_train.columns,'importance':CART_Class.feature_importances_})
important_features.sort_values(by = 'importance',ascending = False,inplace =True)
important_features['cum_importance'] = np.cumsum(important_features['importance'])
important_features.head(10)

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
model = BaggingClassifier(base_estimator=decision_tree, n_estimators=200, random_state=123)
tf_tree = model.fit(X_train, Y_train)

pred = tf_tree.predict(X_test)
print('Prediction accuracy of the model in the test set:',metrics.accuracy_score(Y_test, pred))
print('Evaluation report of the model：\n',metrics.classification_report(Y_test, pred))

In [ ]:
y_score = tf_tree.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)


plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.5, edgecolor = 'black')

plt.plot(fpr, tpr, color='black', lw = 1)

plt.plot([0,1],[0,1], color = 'red', linestyle = '--')

plt.text(0.5,0.3,'ROC curve (area = %0.2f)' % roc_auc)

plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')

plt.show()
roc_auc

In [ ]:
y_score = tf_tree.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)
plt.figure(figsize=(8, 8))
plt.title('Validation ROC')
plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.3, edgecolor = 'black')
plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
plt.ylim(0,1)
plt.xlim(0,1)
plt.legend(loc='best')
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.plot([0,1],[0,1],'r--')
plt.show()

In [ ]:

from sklearn import ensemble

RF_class = ensemble.RandomForestClassifier(n_estimators=90,random_state=123)

RF_class.fit(X_train, Y_train)

RFclass_pred = RF_class.predict(X_test)

print('Prediction accuracy of the model in the test set：\n',metrics.accuracy_score(Y_test, RFclass_pred))
print('Evaluation report of the model：\n',metrics.classification_report(Y_test, RFclass_pred))

In [ ]:
y_score = RF_class.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)


plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.5, edgecolor = 'black')

plt.plot(fpr, tpr, color='black', lw = 1)

plt.plot([0,1],[0,1], color = 'red', linestyle = '--')

plt.text(0.5,0.3,'ROC curve (area = %0.2f)' % roc_auc)

plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')

plt.show()
roc_auc

In [ ]:
y_score = RF_class.predict_proba(X_test)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)

roc_auc = metrics.auc(fpr,tpr)
plt.figure(figsize=(8, 8))
plt.title('Validation ROC')
plt.stackplot(fpr, tpr, color='steelblue', alpha = 0.3, edgecolor = 'black')
plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
plt.ylim(0,1)
plt.xlim(0,1)
plt.legend(loc='best')
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.plot([0,1],[0,1],'r--')
plt.show()

In [ ]:
importance = pd.Series(RF_class.feature_importances_, index = x_train.columns)
plt.figure(figsize=(8, 16))
importance.sort_values().plot(kind = 'barh')
plt.show()
importance.sort_values(ascending = False).head(10).plot(kind = 'barh')
plt.show()

In [ ]:
import numpy as np

pd.set_option('display.max_columns', 20)

pd.set_option('display.max_rows', 130)
important_features = pd.DataFrame({'feature':x_train.columns,'importance':RF_class.feature_importances_})
important_features.sort_values(by = 'importance',ascending = False,inplace =True)
important_features['cum_importance'] = np.cumsum(important_features['importance'])
important_features.head(10)

In [ ]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2020
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print(
            '========================================================================================'
        )
        print(
            '****************************************** {} ******************************************'
            .format(str(i + 1)))
        print(
            '========================================================================================'
        )
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[
            train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2**5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': 2020,
                'nthread': 28,
                'n_jobs': 24,
                'silent': True,
                'verbose': -1,
            }

            model = clf.train(
                params,
                train_matrix,
                50000,
                valid_sets=[train_matrix, valid_matrix],
                verbose_eval=200,
                early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(
                test_x, num_iteration=model.best_iteration)

            # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])

        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x, label=trn_y)
            valid_matrix = clf.DMatrix(val_x, label=val_y)

            params = {
                'booster': 'gbtree',
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'gamma': 1,
                'min_child_weight': 1.5,
                'max_depth': 5,
                'lambda': 10,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'colsample_bylevel': 0.7,
                'eta': 0.04,
                'tree_method': 'exact',
                'seed': 2020,
                'nthread': 36,
                "silent": True,
            }

            watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

            model = clf.train(
                params,
                train_matrix,
                num_boost_round=50000,
                evals=watchlist,
                verbose_eval=200,
                early_stopping_rounds=200)
            val_pred = model.predict(
                valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(
                test_x, ntree_limit=model.best_ntree_limit)

        if clf_name == "cat":
            params = {
                'learning_rate': 0.05,
                'depth': 5,
                'l2_leaf_reg': 10,
                'bootstrap_type': 'Bernoulli',
                'od_type': 'Iter',
                'od_wait': 50,
                'random_seed': 11,
                'allow_writing_files': False
            }

            model = clf(iterations=20000, **params)
            model.fit(
                trn_x,
                trn_y,
                eval_set=(val_x, val_y),
                cat_features=[],
                use_best_model=True,
                verbose=500)

            val_pred = model.predict(val_x)
            test_pred = model.predict(test_x)

        train[valid_index] = val_pred
        test = test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))

        print(cv_scores)

    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

In [ ]:
def lgb_model(x_train, y_train, x_test):
    lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb")
    return lgb_train, lgb_test

def xgb_model(x_train, y_train, x_test):
    xgb_train, xgb_test = cv_model(xgb, x_train, y_train, x_test, "xgb")
    return xgb_train, xgb_test

def cat_model(x_train, y_train, x_test):
    cat_train, cat_test = cv_model(CatBoostRegressor, x_train, y_train, x_test, "cat")

In [ ]:
lgb_train, lgb_test = lgb_model(x_train, y_train, x_test)

In [ ]:
xgb_train, xgb_test = xgb_model(x_train, y_train, x_test)

In [ ]:
cat_train, cat_test = cat_model(x_train, y_train, x_test)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import minmax_scale

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, LSTM, BatchNormalization
from tensorflow.keras.utils import to_categorical 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import os
import librosa
import librosa.display
import glob 
from tqdm import tqdm
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from keras import regularizers

from sklearn.utils import class_weight

In [ ]:
import matplotlib.pyplot as plt


def show(epochs_range, train_loss, val_loss, train_accuracy, val_accuracy, name):

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, train_accuracy, label='Training Accuracy')
    plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.savefig(name)
    plt.show()

In [ ]:
x_train_ed = x_train.values.reshape(-1, 15, 5, 1)
x_test_ed = x_test.values.reshape(-1, 15, 5, 1)
input_dim = (15, 5, 1)
nfold = 10
kf = KFold(n_splits=nfold, shuffle=True, random_state=2020)
prediction1 = np.zeros((1750,20 ))
print(prediction1.shape)
i = 0
for train_index, valid_index in kf.split(x_train_ed, y_train):
    print("\nFold {}".format(i + 1))
    train_x, val_x = x_train_ed[train_index],x_train_ed[valid_index]
    train_y, val_y = y_train[train_index],y_train[valid_index]
    train_x = train_x.reshape(-1, 15, 5, 1)
    val_x = val_x.reshape(-1, 15, 5, 1)
#     print(train_x.shape)
#     print(val_x.shape)
#     train_y = to_categorical(train_y)
#     val_y = to_categorical(val_y)
#     print(train_y.shape)
#     print(val_y.shape)
    model = Sequential()
    model.add(Conv2D(64, (5, 5),padding = "same", input_shape=input_dim, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3),padding = "same",activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(256, (3, 3),padding = "same",activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='Adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    model.summary()
    epochs = 180
    history = model.fit(train_x, train_y, epochs = epochs, batch_size = 64, validation_data = (val_x, val_y))
    
    test_scores = model.evaluate(X_test, Y_test)
    epochs_range = range(1, epochs+1)
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    test_loss = test_scores[0]
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    test_acc = test_scores[1]

    plot_model(model, to_file="model.png",show_shapes=True);
    show(epochs_range, train_loss, val_loss, train_acc, val_acc, 'Model_score_v1')
    print('')
    print('train loss:', train_loss[-1], '   ', 'train accuracy:', train_acc[-1])
    print('val loss:', val_loss[-1], '   ', 'val accuracy:', val_acc[-1])
    print('test loss:', test_loss, '   ', 'test accuracy:', test_acc)
    print('')
    # X_test = np.vstack(X_test)
    predictions = model.predict(x_test_ed.reshape(-1, 15, 5, 1))
    print(predictions.shape)
    prediction1 += ((predictions)) / nfold
    i += 1